https://signate.jp/competitions/567/tutorials/39

# 1. 準備

### 1.1. 基本設定

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #seabornない人はpip installしてね
import os
from datetime import datetime
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.dates as mdates
from sklearn.preprocessing import StandardScaler

# カレントディレクトリを.pyと合わせるために以下を実行
if Path.cwd().name == "notebook":
    os.chdir("..")

# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

# 浮動小数点数を小数点以下3桁で表示するように設定
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
plt.rcParams['font.family'] = 'Meiryo'

### 1.2. csv読み込み

In [ ]:
# point_history.csvの読み込み
df_point_history_sorce = pd.read_csv('data/input/point_history_cleansing.csv')

In [ ]:
# ユーザー基本情報の読み込み
df_user_base_sorce = pd.read_csv("data/input/user_info_merged.csv")

### 1.3. データクレンジング

#### 1.3.1. df_user_base(ユーザ基本情報)のクレンジング

In [ ]:
# DataFrameのコピーを作成
feature_list = [
    'id',
    'club_coin',
    'recycle_point',
    'total_recycle_amount',
    'recycle_amount_per_year',
    'recycle_amount_after_gold_member',
    'rank_continuation_class',
    'gender',
    '緯度',
    '経度',
    '登録日時',
    'カード登録',
    '最終利用日',
    '登録店舗との距離',
    '毎月平均リサイクル量',
    '毎月平均リサイクル回数',
    '毎月平均クラブコインの使用量',
    '毎月平均ガチャの取得量',
    '毎月平均ガチャの使用量',
    '平均rank',
    'サービス利用開始からの経過日数',
    'birthday'
    ]

df_user_base = df_user_base_sorce.copy()
df_user_base = df_user_base[feature_list]

# 紛らわしい列名を改名
df_user_base = df_user_base.rename(columns={'登録日時': 'アプリ登録日時', '最終利用日': 'アプリ最終利用日'})

# objectをdatetimeに変更
df_user_base['アプリ登録日時'] = pd.to_datetime(df_user_base['アプリ登録日時'], errors='coerce')
df_user_base['アプリ最終利用日'] = pd.to_datetime(df_user_base['アプリ最終利用日'], errors='coerce')
df_user_base['カード登録'] = pd.to_datetime(df_user_base['カード登録'], errors='coerce')
df_user_base['アプリ最終利用日'] = pd.to_datetime(df_user_base['アプリ最終利用日'], errors='coerce')
df_user_base['birthday'] = pd.to_datetime(df_user_base['birthday'], errors='coerce')

# 6歳未満(1543個)と100歳以上(12個)を削除
df_user_base = df_user_base[ (df_user_base['birthday'] < pd.to_datetime('2017-01-01')) & (df_user_base['birthday'] > pd.to_datetime('1924-01-01'))]

# df_user_baseに"age"と"age_group"のカラムを追加
df_user_base['age'] = pd.Timestamp.now().year - df_user_base['birthday'].dt.year    # ageの算出・追加

# 今回使用しない可能性が高いカラムは削除
df_user_base = df_user_base.sort_values(by='アプリ登録日時')

#### 1.3.2. df_point_history(point_history.csv)のクレンジング

## TODO:　store_latitude,store_longitudeと、userの緯度経度から、利用店舗との距離を算出してカラムに追加する

In [ ]:
# DataFrameのコピーを作成
df_point_history = df_point_history_sorce.copy()

# objectをdatetimeに変更
df_point_history['use_date'] = pd.to_datetime(df_point_history['use_date'], errors='coerce')

feature_list_point = [
    'user_id',
    'super',
    'status',
    'shop_name_1',
    'amount_kg',
    'rank_id',
    'use_date',
    'store_latitude',
    'store_longitude',
    ]
df_point_history = df_point_history[feature_list_point]
df_point_history = df_point_history.sort_values(by='use_date')

# statusが1以外は削除
df_point_history = df_point_history[df_point_history['status'] == 1]

# amount_kgが0以下は削除
df_point_history = df_point_history[df_point_history['amount_kg'] > 0]


#### 1.3.3. 分析に必要なカラムの作成

継続利用期間（point_historyのuse_date列からRPS最終利用日を抽出したver.）　231228 norosen

In [ ]:
# 各利用者id に対して「RPS利用開始日」「RPS最終利用日」を抽出
first_entries_RPS = df_point_history.groupby('user_id').first().reset_index()
last_entries_RPS = df_point_history.groupby('user_id').last().reset_index()

In [ ]:
# df_user_baseに利用開始日をマージ
df_user_base = pd.merge(df_user_base, first_entries_RPS[['user_id', 'use_date']], left_on='id', right_on='user_id', how='left')
df_user_base = df_user_base.rename(columns={'use_date':'RPS利用開始日'})

# df_user_baseに最終利用日をマージ
df_user_base = pd.merge(df_user_base, last_entries_RPS[['user_id', 'use_date']], left_on='id', right_on='user_id', how='left')
df_user_base = df_user_base.rename(columns={'use_date':'RPS最終利用日'})


df_user_base['RPS利用開始日'] = pd.to_datetime(df_user_base['RPS利用開始日'], errors='coerce')
df_user_base['RPS最終利用日'] = pd.to_datetime(df_user_base['RPS最終利用日'], errors='coerce')

In [ ]:
df_user_base = df_user_base.drop(columns=['user_id_x', 'user_id_y'])

In [ ]:
# RPS継続利用期間を計算
df_user_base['RPS継続利用期間(月)'] = (df_user_base['RPS最終利用日'] - df_user_base['RPS利用開始日']).dt.days / 30  # 月単位で計算
df_user_base = df_user_base[df_user_base['RPS継続利用期間(月)'] >= 0]

#### 1.3.4. マージ

In [ ]:
df_point_history.head(1)

In [ ]:
monthly_grouped_point = df_point_history.groupby(['user_id', df_point_history['use_date'].dt.to_period('M')])['amount_kg'].sum()
monthly_grouped_point = monthly_grouped_point.reset_index()

In [ ]:
# 全ユーザーに対して、カバーすべき年月の範囲を特定します。
date_range = pd.period_range(monthly_grouped_point['use_date'].min(), monthly_grouped_point['use_date'].max(), freq='M')

# 全ユーザーIDを取得します。
user_ids = monthly_grouped_point['user_id'].unique()

# 全てのユーザーIDと年月の組み合わせを持つDataFrameを作成します。
all_combinations = pd.MultiIndex.from_product([user_ids, date_range], names=['user_id', 'use_date'])

# この新しいDataFrameを元のDataFrameとマージします。これにより、元になかった年月の組み合わせはNaNで埋められます。
expanded_df = pd.DataFrame(index=all_combinations).reset_index()
expanded_df = expanded_df.merge(monthly_grouped_point, on=['user_id', 'use_date'], how='left')

# NaNを0で埋めます。
expanded_df['amount_kg'] = expanded_df['amount_kg'].fillna(0)

# 最後に'YearMonth'の形式を'YYYY-MM'に戻します。
expanded_df['use_date'] = expanded_df['use_date'].dt.strftime('%Y-%m')

In [ ]:
merged_df = pd.merge(expanded_df, df_user_base,  left_on='user_id', right_on='id', how='inner')

In [ ]:
merged_df['use_date'] = pd.to_datetime(merged_df['use_date'])
merged_df['use_year'] = merged_df['use_date'].dt.year
merged_df['use_month'] = merged_df['use_date'].dt.month
merged_df['day_name'] = merged_df['use_date'].dt.day_name()
merged_df = merged_df.drop(columns = ['user_id',
                                      'birthday',
                                      'use_date',
                                      'RPS利用開始日',
                                      'RPS最終利用日',
                                      'アプリ最終利用日',
                                      'アプリ登録日時',
                                      'カード登録'                                      
                                     ])

In [ ]:
merged_df = pd.get_dummies(merged_df,columns=['gender','day_name'])

In [ ]:
merged_df = merged_df.astype(float)

In [ ]:
first_columns = ['use_year', 'use_month', 'id', 'amount_kg']

# first_columns に含まれていないカラムを抽出
remaining_columns = [col for col in merged_df.columns if col not in first_columns]

# 新しいカラムの順序を生成
new_columns_order = first_columns + remaining_columns

# DataFrameのカラムを新しい順序で再配置
merged_df = merged_df[new_columns_order]

In [ ]:
merged_df.head(50)

In [ ]:
merged_df.info()

# 2. 予測

In [ ]:
#学習用のデータフレームを作成
dataset = merged_df.copy()
# #評価用のデータフレームを作成(使用するモデルの関係上、前日のデータが必要なため2014-08-31から取得)
# evaluation_dataset_df = merged_df[merged_df["date"]>="2014-08-31"]

In [ ]:
dataset = dataset.sort_values(['use_year','use_month','id'],ascending = True)

In [ ]:
# StandardScalerのインスタンスを作成
scaler = StandardScaler()

# データフレームの全列を標準化
# ここでは、ID列など、標準化不要な列は除外する必要があります
columns_to_scale = dataset.columns.difference(['id', 'use_year', 'use_month',])
dataset[columns_to_scale] = scaler.fit_transform(dataset[columns_to_scale])